# Remove local background

This section draws heavily from the Photutils documentation: https://photutils.readthedocs.io/en/stable/aperture.html

In [ ]:
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt

from photutils.datasets import load_star_image
from photutils.detection import DAOStarFinder
from photutils.aperture import (CircularAperture, CircularAnnulus, 
                                aperture_photometry, ApertureStats)

from astropy.visualization import SqrtStretch
from astropy.visualization.mpl_normalize import ImageNormalize
from astropy.stats import sigma_clipped_stats, SigmaClip


## First, identify sources

This is identical to notebook 01-02

In [ ]:
# photutils allows us to easily access one of their example images
hdu = load_star_image()  
# We will only consider a small portion of the image, for faster processing
data = hdu.data[0:401, 0:401]  

# Plot the image with a square-root normalization
norm = ImageNormalize(stretch=SqrtStretch())
plt.imshow(data, norm=norm, origin='lower', cmap='Greys_r')

In [ ]:
# Use sigma-clipped statistics to estimate the background & background noise
clipped_mean, clipped_med, clipped_std = sigma_clipped_stats(data, sigma=3.0) 
print(clipped_mean, clipped_med, clipped_std)

In [ ]:
daofind = DAOStarFinder(fwhm=3.0, threshold=5.*clipped_std)  
sources = daofind(data - clipped_med)  
for col in sources.colnames:  
    if col not in ('id', 'npix'):
        sources[col].info.format = '%.2f'  # for consistent table output
sources.pprint(max_width=76)  

## Find the flux from one star

In [ ]:
# Select the position of one star
# Using "asarray" produces a regular array containing floats
position = np.asarray([sources[35]["xcentroid"],sources[35]["ycentroid"]],dtype="float64")
print(position)

In [ ]:
# Define our aperture, centered on the star's position, with a radius of three pixels
aperture = CircularAperture(position, r=3.0)

In [ ]:
phot_table = aperture_photometry(data, aperture)

In [ ]:
phot_table

Note that this just gives us the sum of the counts within our defined aperture. This includes the background!

To calculate the background level, we need to define an annulus around our target position

In [ ]:
annulus_aperture = CircularAnnulus(position, r_in=7, r_out=12)
bkgd_table = aperture_photometry(data, annulus_aperture)

In [ ]:
bkgd_table

Our background counts are higher than our source counts! This should not be surprising, though, since the background annulus has many more pixels.

To calculate the area within an aperture, we use the `.area_overlap` method, which compares the aperture to the image and returns the number of good (i.e., non-masked) pixels within the aperture

In [ ]:
source_area = aperture.area_overlap(data)
bkgd_area = annulus_aperture.area_overlap(data)

In [ ]:
print(source_area,bkgd_area)

In [ ]:
# Calculate the background counts per pixel
bkgd_per_pixel = bkgd_table["aperture_sum"][0] / bkgd_area

In [ ]:
# Determine the total background counts within the source aperture
bkgd_in_source = bkgd_per_pixel * source_area

In [ ]:
# Subtract the background from the source counts
source_counts = phot_table["aperture_sum"][0] - bkgd_in_source
print(source_counts)

Nearly half of the counts within our source aperture are actually background counts!

In [ ]:
# Let's plot just the region around our star, and show our apertures
norm = ImageNormalize(stretch=SqrtStretch())
plt.imshow(data, norm=norm, origin='lower', cmap='Greys_r',zorder=-5)

ap_patches = aperture.plot(color='red', lw=2,
                           label='Photometry aperture',zorder=1)
ann_patches = annulus_aperture.plot(color='yellow', lw=2,
                                    label='Background annulus',zorder=2)
handles = (ap_patches[0], ann_patches[0])
plt.legend(loc=(0.17, 0.05), facecolor='k', labelcolor='white',
           handles=handles, prop={'weight': 'bold', 'size': 11})

plt.xlim(120,180)
plt.ylim(55,105)

### Sigma clipping

In the image above, we can see that our background annulus just barely overlaps with the image of a nearby star. This is not uncommon, particularly in crowded fields. 

We may also encounter things like cosmic rays in the background annulus, which can inflate our background count despite not reflecting the true background levels. Therefore, it is more common to compute a sigma-clipped median of the background counts per pixel.

The `ApertureStats` class will handle some of the computations that we did by hand in the previous section. It will compute several statistics about the pixels within the aperture.

In [ ]:
sigclip = SigmaClip(sigma=3.0, maxiters=10)

# We don't want to do any sigma clipping on our photometry / source aperture
aper_stats = ApertureStats(data, aperture, sigma_clip=None)

# We do want sigma clipping when analyzing the background annulus
bkg_stats = ApertureStats(data, annulus_aperture, sigma_clip=sigclip)

In [ ]:
# We want the median value
print(f"From the previous section: {bkgd_per_pixel:.2f}")
print("Sigma clipped:",bkg_stats.median)

In [ ]:
# As before, we multiply the median background value by the number of 
# pixels in the photometry aperture
total_bkg = bkg_stats.median * aper_stats.sum_aper_area.value
print(total_bkg) 

In [ ]:
# Then the background-subtracted counts for our source is given by
apersum_bkgsub = aper_stats.sum - total_bkg
print(apersum_bkgsub)  

In [ ]:
# Finally, let's recompute our statistics on the source, accounting for the 
# local background values. 
# ApertureStats takes a local_bkg keyword that accepts the per-pixel value

aper_stats_bkgsub = ApertureStats(data, aperture,
                                  local_bkg=bkg_stats.median)

# The total counts within the aperture should match what we calculated 
# in the previous cell
print(aper_stats_bkgsub.sum)

## Your turn - Subtract the background from all of the sources in our list

Hint: your first step is to create a list or array of (x, y) pairs. You can provide that list or array to the Aperture classes and it will produce apertures at every location. See notebook 01-02 for an example.

Consult the Photutils documentation and you'll see that all of the apertures can accept a list or array of positions. Then other functions will run on every position identified in the aperture object. 